In [7]:
import ROOT
import os
import glob
import dask.dataframe as dd
import iy10_ultrashower_utilities as shower
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz

# Defining data structure
class TxtFile:
    def __init__(self):
        self.column1 = 0
        self.column2 = 0
        self.column3 = 0
        self.column4 = 0
        self.column5 = 0
        self.column6 = 0.0
        self.column7 = 0.0
        self.column8 = 0.0
        self.column9 = 0.0
        self.column10 = 0.0
        self.column11 = 0.0
        self.column12 = 0

# Function for creating and saving histogram
def Histogram(df, column, output_path, bins, xmin, xmax):
    
    if len(df.columns) == 0:     #To avoid files with no data
        print("Dataframe is empty. Skipping histogram creation.")
        return

    canvas = ROOT.TCanvas("canvas", "Asymmetry_distribution", 800, 600)
    dask_hist = df[column].compute()

    pad = ROOT.gPad
    root_hist = ROOT.TH1F("1D Histogram", f"{column} Distribution", bins, xmin, xmax)

    for value in dask_hist:
        root_hist.Fill(value)

    root_hist.SetFillColor(ROOT.kGreen)
    root_hist.GetXaxis().SetTitle(column)
    root_hist.GetYaxis().SetTitle("Counts")
    root_hist.GetXaxis().CenterTitle()
    root_hist.GetYaxis().CenterTitle()
    root_hist.Draw("BAR")
    
    canvas.SaveAs(os.path.join(output_path, f"{column} Distribution.png"))

    del root_hist  # Deleting the ROOT histogram object after saving the plot
    canvas.Close()  # Closing the canvas for memory issue
    
def Histogram2D(df, x_column, y_column, output2D_path, xbins, xmin, xmax, ybins, ymin, ymax):
    
    if len(df.columns) == 0:
        print("Dataframe is empty. Skipping histogram creation.")
        return
    
    canvas = ROOT.TCanvas("canvas2D", "2D Histogram", 800, 600)
    x_dask = df[x_column].compute() #importing data into dask dataframe
    y_dask = df[y_column].compute()
    
    pad = ROOT.gPad
    hist = ROOT.TH2F("2D Histogram", f"{y_column} vs {x_column} Distribution", xbins, xmin, xmax, ybins, ymin, ymax) 
    
    for x_value,y_value in zip(x_dask,y_dask):
        hist.Fill(x_value,y_value)
    
    hist.GetXaxis().SetTitle(x_column)
    hist.GetYaxis().SetTitle(y_column)
    hist.GetXaxis().CenterTitle()
    hist.GetYaxis().CenterTitle()
    hist.Draw("COLZ")
    #file2D_name = os.path.splitext(os.path.basename(file_path))[0]
    canvas.SaveAs(os.path.join(output2D_path, f"{y_column}_vs_{x_column} Distribution.png"))
    
    del hist
    canvas.Close()    

# Directory where txt files are located
file_list = glob.glob("/home/aklima/Muon_Shower_Data/TextFiles/*_EventInfo.txt")

# Directory to save histograms
output_path = "/home/aklima/Muon_Shower_Data/Combined_histograms"

# Process files in batches to avoid memory issues and combine all datafiles into one
batch_size = 10
dfs = []
for n in range(0, len(file_list), batch_size):
    batch_file_path = file_list[n:n+batch_size]
    for file_path in batch_file_path:
        df = dd.read_csv(file_path, delimiter='\t', comment='#', names=["Run","Subrun","Event","SubStart","SubStop","Occupancy_XZ","Occupancy_YZ","AsymXZ","AsymYZ","Angle_XZ","AngleYZ","Timestamp"])

        if len(df.columns) > 0:
            dfs.append(df)

combined_df = dd.concat(dfs)

# Plot, plot and plot because the game is on Watson!
Histogram(combined_df, 'AsymXZ', output_path, 100, 0, 0.5)
Histogram(combined_df, 'AsymYZ', output_path, 100, 0, 0.5)
Histogram(combined_df, 'Occupancy_XZ', output_path, 100, 0, 0.75)
Histogram(combined_df, 'Occupancy_YZ', output_path, 100, 0, 0.75)
Histogram2D(combined_df, 'AsymYZ', 'Occupancy_YZ', output_path, 100, 0, 1, 100, 0, 1)
Histogram2D(combined_df, 'AsymXZ', 'Occupancy_XZ', output_path, 100, 0, 1, 100, 0, 1)
Histogram2D(combined_df, 'AsymXZ', 'AsymYZ', output_path, 100, 0, 0.5, 100, 0, 0.5)
Histogram2D(combined_df, 'Occupancy_XZ', 'Occupancy_YZ', output_path, 100, 0, 0.75, 100, 0, 0.75)

Info in <TCanvas::Print>: png file /home/aklima/Muon_Shower_Data/Combined_histograms/AsymXZ Distribution.png has been created
Info in <TCanvas::Print>: png file /home/aklima/Muon_Shower_Data/Combined_histograms/AsymYZ Distribution.png has been created
Info in <TCanvas::Print>: png file /home/aklima/Muon_Shower_Data/Combined_histograms/Occupancy_XZ Distribution.png has been created
Info in <TCanvas::Print>: png file /home/aklima/Muon_Shower_Data/Combined_histograms/Occupancy_YZ Distribution.png has been created
Info in <TCanvas::Print>: png file /home/aklima/Muon_Shower_Data/Combined_histograms/Occupancy_YZ_vs_AsymYZ Distribution.png has been created
Info in <TCanvas::Print>: png file /home/aklima/Muon_Shower_Data/Combined_histograms/Occupancy_XZ_vs_AsymXZ Distribution.png has been created
Info in <TCanvas::Print>: png file /home/aklima/Muon_Shower_Data/Combined_histograms/AsymYZ_vs_AsymXZ Distribution.png has been created
Info in <TCanvas::Print>: png file /home/aklima/Muon_Shower_Data